In [ ]:
import polyfempy as pf
import json
import numpy as np
import torch

torch.set_default_dtype(torch.float64)

In [ ]:
# Differentiable simulator that computes shape derivatives
class Simulate(torch.autograd.Function):

    @staticmethod
    def forward(ctx, solver, vertices):
        solver.mesh().set_vertices(vertices)
        solver.set_cache_level(pf.CacheLevel.Derivatives) # enable backward derivatives
        solver.solve()
        cache = solver.get_solution_cache()
        solutions = torch.zeros((solver.ndof(), cache.size()))
        for t in range(cache.size()):
            solutions[:, t] = torch.tensor(cache.solution(t))
        
        ctx.solver = solver
        return solutions

    @staticmethod
    def backward(ctx, grad_output):
        ctx.solver.solve_adjoint(grad_output)
        return None, torch.tensor(pf.shape_derivative(ctx.solver))

In [ ]:
root = "../data/differentiable/input"
with open(root + "/initial-contact.json", "r") as f:
    config = json.load(f)

config["root_path"] = root + "/initial-contact.json"

solver = pf.Solver()
solver.set_settings(json.dumps(config), False)
solver.set_log_level(2)
solver.load_mesh_from_settings()

mesh = solver.mesh()
v = mesh.vertices()
vertices = torch.tensor(solver.mesh().vertices(), requires_grad=True)

In [ ]:
# Verify gradient

def loss(vertices):
    solutions = Simulate.apply(solver, vertices)
    return torch.linalg.norm(solutions[:, -1])

torch.set_printoptions(12)

param = vertices.clone().detach().requires_grad_(True)
theta = torch.randn_like(param)
l = loss(param)
l.backward()
grad = param.grad
t = 1e-6
with torch.no_grad():
    analytic = torch.tensordot(grad, theta)
    f1 = loss(param + theta * t)
    f2 = loss(param - theta * t)
    fd = (f1 - f2) / (2 * t)
    print(f'grad {analytic}, fd {fd} {(f1 - l) / t} {(l - f2) / t}, relative err {abs(analytic - fd) / abs(analytic):.3e}')
    print(f'f(x+dx)={f1}, f(x)={l.detach()}, f(x-dx)={f2}')
    assert(abs(analytic - fd) <= 1e-4 * abs(analytic))